In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 4.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=0ca75b1b57f434c8ec32f4e050ce6542eff8cabc761311132c1a91b6be53b7a4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-66urygsq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-66urygsq
     |████████████████████████████████| 129 kB 4.0 MB/s 
     |████████████████████████████████| 54.7 MB 29 kB/s 
     |████████████████████████████████| 4.5 MB 55.1 MB/s 
     |████████████████████████████████| 1.2 MB 66.5 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x39542000 @  0x7f5bd9c8d615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd

In [ ]:
musinsa_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/미니프로젝트/res/musinsa2_csv')

In [ ]:
musinsa_data['result']= musinsa_data['sex']+" "+musinsa_data['height']+" "+musinsa_data['weight']

In [ ]:
musinsa_data.loc[(musinsa_data['size'] == "S"), 'size'] = 0  #S => 0
musinsa_data.loc[(musinsa_data['size'] == "M"), 'size'] = 1  #M => 1
musinsa_data.loc[(musinsa_data['size'] == "L"), 'size'] = 2  #L => 2
musinsa_data.loc[(musinsa_data['size'] == "XL"), 'size'] = 3  #XL => 3

In [ ]:
musinsa_data

,Unnamed: 0,sex,height,weight,clothname,size,review,result
0,0,남성,177cm,88kg,엔젤 와펜 집업 후드 그레이,3,두께감도있고 꽤 오버핏에 편하게 입기 좋아요~무난하게 입을듯,남성 177cm 88kg
1,1,남성,185cm,74kg,엔젤 와펜 집업 후드 그레이,3,생각했던것보다 핏이 더 좋게 나옵니다. 로고도 귀여워요 :),남성 185cm 74kg
2,2,남성,168cm,65kg,엔젤 와펜 집업 후드 그레이,1,간절기 말고 초겨울에도 따뜻하게 입을 수 있을거 같습니다.생각보다 두꺼워요,남성 168cm 65kg
3,3,여성,168cm,70kg,엔젤 와펜 집업 후드 블랙,2,사이즈도 좋고 여러옷에 다 잘맞아요.와릿이즌 옷은 실패한적이 없어요.딱 좋아요.이쁜...,여성 168cm 70kg
4,4,남성,168cm,52kg,엔젤 와펜 집업 후드 그레이,1,진짜이뻐요 로고랑 뒤에 프린팅도 이쁘고 자주자주입을거같아요,남성 168cm 52kg
...,...,...,...,...,...,...,...,...
13236,13236,남성,175cm,75kg,엔젤 와펜 집업 후드 그레이,2,옷도 이쁘고 질도 괜찮은데 후드끈이 없이 배송이 됬네요 비싼돈주고 샀는데 후드끈이 ...,남성 175cm 75kg
13237,13237,남성,175cm,75kg,엔젤 와펜 집업 후드 그레이,1,이쁘고 기모 있어요 오버하게 입기 좋은것 같아요,남성 175cm 75kg
13238,13238,남성,175cm,75kg,엔젤 와펜 집업 후드 그레이,0,배송이 정말 느리네요... 9월2일에 주문했는데 9월 7일에 왔어요ㅠ,남성 175cm 75kg
13239,13239,남성,175cm,75kg,엔젤 와펜 집업 후드 블랙,0,키가 큰 편이라 사이즈 s 사면서 혹시 작을까 걱정했는데 하나도 안 작고 넉넉해요 ...,남성 175cm 75kg


In [ ]:
data_list = []
for q, label in zip(musinsa_data['result'], musinsa_data['size'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(len(data_list))
print(data_list[0])
print(data_list[6000])
print(data_list[12000])

13241
['남성  177cm  88kg', '3']
['남성  171cm  61kg', '1']
['남성 175cm 75kg', '3']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

9930
3311
['남성  174cm  65kg', '1']
['남성 175cm 75kg', '2']


In [ ]:

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [1]:
## Setting parameters
max_len = 64  # 텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

## 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다

### BERTSentenceTransform로 토큰화와 패딩을 실행

In [ ]:

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1419,  546,  157,  384,  620,  417,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(8, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:

data_test[1]

(array([   2, 1419,  529,  214,  384,  529,  105,  417,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

### BERT 모델 불러오기

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/156 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.007206678390503 train acc 0.140625
epoch 1 train acc 0.33383413461538464


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/52 [00:00<?, ?it/s]

...

epoch 9 test acc 0.503401186579378


  0%|          | 0/156 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 1.1335831880569458 train acc 0.484375
epoch 10 train acc 0.5223557692307692


  0%|          | 0/52 [00:00<?, ?it/s]

epoch 10 test acc 0.5047117941898527


# 모델 저장하기

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/미니프로젝트/res/')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/미니프로젝트/res'

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/미니프로젝트/res/'
torch.save(model, path + 'size_model.pt')  # 전체 모델 저장

In [ ]:
torch.save(model.state_dict(), 'size_model_state_dict.pt')  # 모델 객체의 state_dict 저장

In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'size_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
#★★★현재경로가 model이 있는 폴더여야함★★★
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/미니프로젝트/res/')

model1 = torch.load('size_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('size_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('size_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

# 기본형(정돈)

In [2]:

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        print('추천사이즈는 ')
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("S")
            elif np.argmax(logits) == 1:
                test_eval.append("M")
            elif np.argmax(logits) == 2:
                test_eval.append("L")
            elif np.argmax(logits) == 3:
                test_eval.append("XL")
            
        print(test_eval[0] + "입니다.")

In [3]:
#질문 무한반복하기!
end = 1
while end == 1 :
    sentence = input("말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

말을 입력해주세요 : 여성 163cm 45kg


NameError: ignored